### Single body formulation

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown
# import os # Uncomment if you want to use a subdirectory for figures

# Global variable to store the current figure for saving
current_fig = None

def hamiltonian(epsilon1, epsilon2, epsilon3, t, Delta):
    """
    Defines the 3-site Bogoliubov-de Gennes Hamiltonian matrix.
    This specific structure leads to det(H)=0 for N=3 sites if Delta_p is antisymmetric.
    Parameters:
    epsilon1, epsilon2, epsilon3: On-site energies for sites 1, 2, and 3.
    t: Hopping parameter.
    Delta: Pairing strength parameter.
    """
    d = Delta  # Use 'd' as a shorthand for Delta in the matrix
    # H_p is the particle block (real symmetric)
    # Delta_p is the pairing block (real antisymmetric for N=3, leading to det(H)=0)
    return np.array([[ epsilon1,        t,          0,          0,         d,          0],
                     [        t, epsilon2,          t,          -d,          0,         d],
                     [        0,        t,   epsilon3,          0,          -d,          0],
                     [        0,        -d,          0,  -epsilon1,         -t,          0], # Delta_p^dagger = Delta_p^T = -Delta_p
                     [       d,        0,          -d,         -t,  -epsilon2,         -t], # (since Delta_p is real antisymmetric)
                     [        0,       d,          0,          0,         -t,  -epsilon3]  # -H_p^* = -H_p (since H_p is real)
])


def compute_eigensystem(epsilon1, epsilon2, epsilon3, t, Delta):
    """
    Computes eigenvalues and eigenvectors for the given Hamiltonian parameters.
    Parameters:
    epsilon1, epsilon2, epsilon3: On-site energies for sites 1, 2, and 3.
    t: Hopping parameter.
    Delta: Pairing strength parameter.
    Returns:
    Sorted eigenvalues (real part) and corresponding eigenvectors (real part).
    """
    H = hamiltonian(epsilon1, epsilon2, epsilon3, t, Delta)
    eigvals, eigvecs = np.linalg.eig(H)
    idx = np.argsort(eigvals)  # Sort eigenvalues and corresponding eigenvectors
    return np.real(eigvals[idx]), np.real(eigvecs[:, idx])

def plot_energy_vs_epsilon(t, Delta):
    """
    Plots energy bands as a function of a uniformly varying on-site energy epsilon.
    Also displays eigenvectors for epsilon = 0.
    Parameters:
    t: Hopping parameter.
    Delta: Pairing strength parameter.
    """
    plt.close('all')  # Close previous figures to avoid duplicate plots
    global current_fig

    epsilon_vals = np.linspace(-5, 5, 400)
    # Assuming epsilon1 = epsilon2 = epsilon3 = eps for the sweep
    bands = [compute_eigensystem(eps, eps, eps, t, Delta)[0] for eps in epsilon_vals]
    bands = np.array(bands)

    plt.figure(figsize=(8, 6))
    for i in range(bands.shape[1]):  # bands.shape[1] will be 6 for a 6x6 Hamiltonian
        plt.plot(epsilon_vals, bands[:, i], label=fr'$E_{{{i+1}}}$')

    # Add t and Δ to legend using empty plots with labels
    plt.plot([], [], ' ', label=fr'$t = {t:.2f}$') # Format slider values
    plt.plot([], [], ' ', label=fr'$\Delta = {Delta:.2f}$') # Format slider values
    
    plt.legend(loc='best') 
    # plt.grid(True) # Optional: uncomment for a grid
    plt.xlabel(r'$\varepsilon = \varepsilon_1 = \varepsilon_2 = \varepsilon_3$', fontsize=15)
    plt.ylabel('Energy', fontsize=15)
    plt.xlim(-4, 4)
    plt.ylim(-6, 6)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.title('Energy Bands vs. Site Energy $\\varepsilon$')

    current_fig = plt.gcf()  # Store the current figure
    plt.show()

    # Compute and display the eigenvectors for ε = 0
    eps0 = 0  # Chosen value of epsilon to display eigenvectors
    # Assuming epsilon1 = epsilon2 = epsilon3 = eps0
    eigvals, eigvecs = compute_eigensystem(eps0, eps0, eps0, t, Delta)
    
    eigvec_str = fr"### Eigenvectors at \( \varepsilon_1 = \varepsilon_2 = \varepsilon_3 = {eps0} \), \( t = {t:.2f} \), \( \Delta = {Delta:.2f} \):\n"
    # Display all 6 eigenvalues and eigenvectors
    for i in range(eigvals.shape[0]): 
        eigvec_str += f"**Eigenvalue {eigvals[i]:.3f}:** \n"
        eigvec_str += f"[{', '.join(f'{v:.3f}' for v in eigvecs[:, i])}]\n\n"

    display(Markdown(eigvec_str))  # Display nicely formatted eigenvectors

# Sliders for t and Δ
t_slider = widgets.FloatSlider(min=-5.0, max=5.0, step=0.1, value=1.0, description='$t$')
Delta_slider = widgets.FloatSlider(min=-5.0, max=5.0, step=0.1, value=1.0, description="$\\Delta$")

# Save button
save_button = widgets.Button(description="Save Figure")

def save_figure(_):
    """Saves the current plot to a PDF file."""
    global current_fig
    if current_fig:
        # Ensure the directory exists or save to a known valid path.
        # Saving in the current working directory by default:
        filename = f"plot_t={t_slider.value:.2f}_Delta={Delta_slider.value:.2f}.pdf"
        
        # If you want to use a subdirectory like 'figures_minimal':
        # figures_dir = "figures_minimal"
        # if not os.path.exists(figures_dir):
        #     os.makedirs(figures_dir)
        # filename = os.path.join(figures_dir, f"plot_t={t_slider.value:.2f}_Delta={Delta_slider.value:.2f}.pdf")
        
        current_fig.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Figure saved as {filename}")
    else:
        print("No figure to save. Please generate a plot first.")

save_button.on_click(save_figure)

# Display the interactive widgets
display(widgets.VBox([t_slider, Delta_slider, save_button]))
interactive_plot = widgets.interactive_output(plot_energy_vs_epsilon, {'t': t_slider, 'Delta': Delta_slider})
display(interactive_plot)

Output()

### Many body formulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def many_body_H(epsilon, t, Delta):
    """
    Construct the 4x4 many-body Hamiltonian with ε₁ = ε₂ = ε:
    
        H = [[ 0,       0,     0,     Δ      ],
             [ 0,     ε,      t,      0      ],
             [ 0,      t,     ε,      0      ],
             [ Δ,      0,     0,    2ε      ]]
    """
    return np.array([
        [0,         0,       0,       Delta],
        [0,     epsilon,      t,         0   ],
        [0,         t,   epsilon,       0   ],
        [Delta,      0,       0,    2 * epsilon]
    ])

def compute_eigensystem(epsilon, t, Delta):
    H = many_body_H(epsilon, t, Delta)
    eigvals, eigvecs = np.linalg.eig(H)
    # Sort eigenvalues in ascending order for consistency
    idx = np.argsort(eigvals)
    return np.real(eigvals[idx]), eigvecs[:, idx]

def plot_energy_vs_epsilon(t, Delta):
    epsilon_vals = np.linspace(-5, 5, 400)
    # For each value of ε compute the sorted eigenvalues
    bands = [compute_eigensystem(eps, t, Delta)[0] for eps in epsilon_vals]
    bands = np.array(bands)
    
    plt.figure(figsize=(8, 6))
    # for i in range(bands.shape[1]):
    #     plt.plot(epsilon_vals, bands[:, i], label=rf'$E_{{{i+1}}}$')
    
    plt.plot(epsilon_vals, bands[:, 0], label=rf'$E_1$', color = 'red', linestyle='dashed')
    plt.plot(epsilon_vals, bands[:, 1], label=rf'$E_2$', color = 'red')
    plt.plot(epsilon_vals, bands[:, 2], label=rf'$E_3$', color = 'blue')
    plt.plot(epsilon_vals, bands[:, 3], label=rf'$E_4$', color = 'blue', linestyle='dashed')
    # Add t and Δ info to the legend (using empty plots)
    plt.plot([], [], ' ', label=f't = {t}')
    plt.plot([], [], ' ', label=f'Δ = {Delta}')
    
    # Christain Gutman, Quantum för läkemedel
    # Walse Miller, Photon detectorer, entanglement för vävnad, mikroskopi genom skallbenet, Kvantfysik och applikationer av kvantfysik
    # SU som leds Mohammed Burabani, spinna ut start ups ur forskningen, Jobbar med quantum communiation cryptography
    # 

    #plt.legend()
    #plt.grid(True)

    plt.xlabel(r'$\varepsilon$',fontsize=15)
    plt.ylabel('Energy', fontsize=15)
    plt.xlim(-4, 4)
    plt.ylim(-6, 6)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    #plt.legend(fontsize=12)
    
    global current_fig
    current_fig = plt.gcf()  # Save current figure to allow saving
    plt.show()

# Create sliders for t and Δ
t_slider = widgets.FloatSlider(min=-5.0, max=5.0, step=0.1, value=1.0, description='t')
Delta_slider = widgets.FloatSlider(min=-5.0, max=5.0, step=0.1, value=1.0, description='Δ')

# Create a save button to export the current figure as PDF
save_button = widgets.Button(description="Save Figure")

def save_figure(b):
    if current_fig:
        filename = f"figures_minimal/many_body_plot_t={t_slider.value}_Delta={Delta_slider.value}.pdf"
        current_fig.savefig(filename, dpi=300)
        print(f"Figure saved as {filename}")

save_button.on_click(save_figure)

# Display the interactive widgets
display(widgets.VBox([t_slider, Delta_slider, save_button]))
widgets.interactive_output(plot_energy_vs_epsilon, {'t': t_slider, 'Delta': Delta_slider})

Output()